<a href="https://colab.research.google.com/github/kingyeet9048/final_dsci_project/blob/master/Camshift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [967]:
# import the necessary packages
import numpy as np
import argparse
import cv2 as cv
# initialize the current frame of the video, along with the list of# ROI points along with whether or not this is input mode
frame = None
roiPts = []
inputMode = False

In [964]:
def selectROI(event, x, y, flags, param):
  global frame, roiPts, inputMode

  if inputMode and event == cv2.EVENT_LBUTTONDOWN and len(roiPts) < 4:
    roiPts.append((x, y))
    cv2.circle(frame, (x, y), 4, (0, 255, 0), 2)
    cv2.imshow("frame", frame)

https://www.pyimagesearch.com/wp-content/uploads/2014/11/opencv_crash_course_camshift.pdf

In [965]:

def main():
    camera = cv2.VideoCapture('videos/phone.MOV')
    global frame, roiPts, inputMode
    # setup the mouse callback
    cv2.namedWindow("frame")
    cv2.setMouseCallback("frame", selectROI)

    # initialize the termination criteria for cam shift, indicating
    # a maximum of ten iterations or movement by a least one pixel
    # along with the bounding box of the ROI
    termination = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
    roiBox = None

    # keep looping over the frames
    while True:
        # grab the current frame
        (grabbed, frame) = camera.read()

        # check to see if we have reached the end of the
        # video
        if not grabbed:
            break

        # if the see if the ROI has been computed
        if roiBox is not None:
            # convert the current frame to the HSV color space
            # and perform mean shift
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            backProj = cv2.calcBackProject([hsv], [0], roiHist, [0, 180], 1)

            # apply cam shift to the back projection, convert the
            # points to a bounding box, and then draw them
            (r, roiBox) = cv2.CamShift(backProj, roiBox, termination)
            pts = np.int0(cv2.boxPoints(r))
            cv2.polylines(frame, [pts], True, (0, 255, 0), 2)

        # show the frame and record if the user presses a key
        cv2.imshow("frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # handle if the 'i' key is pressed, then go into ROI
        # selection mode
        if key == ord("a") and len(roiPts) < 4:
            # indicate that we are in input mode and clone the
            # frame
            inputMode = True
            orig = frame.copy()

            # keep looping until 4 reference ROI points have
            # been selected; press any key to exit ROI selction
            # mode once 4 points have been selected
            while len(roiPts) < 4:
                cv2.imshow("frame", frame)
                cv2.waitKey(0)

            # determine the top-left and bottom-right points
            roiPts = np.array(roiPts)
            print(roiPts)
            cv2.waitKey(0)
            s = roiPts.sum(axis = 1)
            tl = roiPts[np.argmin(s)]
            br = roiPts[np.argmax(s)]

            # grab the ROI for the bounding box and convert it
            # to the HSV color space
            roi = orig[tl[1]:br[1], tl[0]:br[0]]
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            #roi = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)

            # compute a HSV histogram for the ROI and store the
            # bounding box
            roiHist = cv2.calcHist([roi], [0], None, [16], [0, 180])
            roiHist = cv2.normalize(roiHist, roiHist, 0, 255, cv2.NORM_MINMAX)
            roiBox = (tl[0], tl[1], br[0], br[1])

        # if the 'q' key is pressed, stop the loop
        elif key == ord("q"):
            break

    # cleanup the camera and close any open windows
    camera.release()
    cv2.destroyAllWindows()


In [966]:
main()

[[770 675]
 [802 675]
 [810 754]
 [780 755]]


In [1090]:
cap  = cv.VideoCapture('videos/phone.MOV')
ret, frame = cap.read()

In [1091]:
r, h, c, w = 640,40,800,20
track_window = (c, r, w, h)

In [1092]:
roi = frame[r:r+h, c:c+w]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0.,60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0,1],mask,[12, 12],[0,181, 0, 256])
#cv.normalize(roi_hist,roi_hist,0, 255,cv.NORM_MINMAX)


In [1093]:
# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )

In [1094]:
while(1):
    ret ,frame = cap.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0, 1],roi_hist,[0,180, 0, 256],1)
        #clean up the noice
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
        dst = cv.filter2D(dst, -1, kernel)
        # apply camshift to get the new location
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        # Draw it on image
        pts = cv.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv.polylines(frame,[pts],True, (255, 0, 0),2)
        cv.imshow("img2", img2)
        k = cv.waitKey(60) & 0xff
        if k == 32:
            break
        else:
            cv.imwrite(chr(k)+".jpg",dst)
    else:
        break
cv.destroyAllWindows()
cap.release()